In [ ]:
from kedro.config import OmegaConfigLoader

loader = OmegaConfigLoader(conf_source="conf/base")
params = loader["parameters"]
print(params["Control_date"]["centaline_estates"])  # Should output "2025-04-09"


In [1]:
# Change CSV to Parquet

import pandas as pd

def csv_to_parquet(csv_file_path, parquet_file_path):
    try:
        # Read the CSV file into a DataFrame
        df = pd.read_csv(csv_file_path)
        
        # Write the DataFrame to a Parquet file
        df.to_parquet(parquet_file_path, index=False)
        
        print(f"Successfully converted {csv_file_path} to {parquet_file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage
csv_file_path = '2025-04-02_centanet_estates_scraped.csv'
parquet_file_path = 'estate_details_raw.parquet'

# Ensure pandas is installed
# pip install pandas

csv_to_parquet(csv_file_path, parquet_file_path)


Successfully converted 2025-04-02_centanet_estates_scraped.csv to estate_details_raw.parquet


In [6]:
import pandas as pd

parquet_file_path = 'data/02_intermediate/estate_listings.parquet'
pq_file = pd.read_parquet(parquet_file_path)
print(pq_file.shape)  
pq_file.head()

(1219, 11)


,Name,Address,Blocks,Units,UnitRate,MoM,ForSale,ForRent,Link,Region,District
0,The Merton,38 New Praya Kennedy Town,3,"1,182","$17,698/ ft²",5.06%,42,4,https://hk.centanet.com/estate/en/The-Merton/2...,Hong Kong Island,All Hong Kong Island
1,Manhattan Heights,28 New Praya Kennedy Town,1,476,--,--,18,2,https://hk.centanet.com/estate/en/Manhattan-He...,Hong Kong Island,All Hong Kong Island
2,Smithfield Terrace,71-77 Smithfield,4,912,"$12,325/ ft²",16.55%,30,8,https://hk.centanet.com/estate/en/Smithfield-T...,Hong Kong Island,All Hong Kong Island
3,Cadogan,37A Cadogan Street,1,197,--,--,15,3,https://hk.centanet.com/estate/en/Cadogan/2-SS...,Hong Kong Island,All Hong Kong Island
4,The Hudson,11 Davis Street,1,134,--,--,12,5,https://hk.centanet.com/estate/en/The-Hudson/2...,Hong Kong Island,All Hong Kong Island
